In [1]:
import pandas as pd

In [2]:
import numpy as np

In [3]:
df = pd.read_csv("C:/Users/pkitt/Downloads/playground-series-s5e6/train.csv")

In [4]:
df.cat =df.select_dtypes(include=[object]) 

C:\Users\pkitt\AppData\Local\Temp\ipykernel_13408\1653857449.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.cat =df.select_dtypes(include=[object])


In [5]:
df.num=df.select_dtypes(include=[np.number])

C:\Users\pkitt\AppData\Local\Temp\ipykernel_13408\610118065.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.num=df.select_dtypes(include=[np.number])


In [6]:
df.cat.head()

,Soil Type,Crop Type,Fertilizer Name
0,Clayey,Sugarcane,28-28
1,Sandy,Millets,28-28
2,Sandy,Millets,17-17-17
3,Sandy,Barley,10-26-26
4,Red,Paddy,DAP


In [7]:
from sklearn.preprocessing import LabelEncoder

In [8]:
le = LabelEncoder()

In [9]:
df.cat["Soil Type"] = le.fit_transform(df.cat["Soil Type"])

In [10]:
df.cat["Crop Type"] = le.fit_transform(df.cat["Crop Type"])

In [11]:
df.cat["Fertilizer Name"] = le.fit_transform(df.cat["Fertilizer Name"])

In [12]:
df.cat.tail(20)

,Soil Type,Crop Type,Fertilizer Name
749980,0,10,5
749981,2,2,2
749982,3,8,0
749983,4,3,2
749984,4,6,1
749985,3,3,0
749986,0,10,3
749987,3,3,6
749988,0,3,1
749989,0,4,4


In [13]:
df_train = pd.concat([df.num,df.cat],axis=1)

In [14]:
matrix=df_train.corr()

In [15]:
matrix

,id,Temparature,Humidity,Moisture,Nitrogen,Potassium,Phosphorous,Soil Type,Crop Type,Fertilizer Name
id,1.000000,-0.001069,-0.000310,0.001752,0.001505,-0.000375,-0.001234,0.002831,-0.000953,-0.000179
Temparature,-0.001069,1.000000,0.001758,-0.001633,-0.006073,-0.009913,0.000298,-0.007621,-0.006753,0.001727
Humidity,-0.000310,0.001758,1.000000,0.003424,0.006502,0.005229,0.002413,0.003921,0.002414,0.010626
Moisture,0.001752,-0.001633,0.003424,1.000000,0.003016,-0.001931,-0.006735,-0.003941,-0.003400,0.011493
Nitrogen,0.001505,-0.006073,0.006502,0.003016,1.000000,0.000543,-0.005464,-0.000650,0.001611,-0.004725
Potassium,-0.000375,-0.009913,0.005229,-0.001931,0.000543,1.000000,0.000113,0.005314,-0.003748,-0.010872
Phosphorous,-0.001234,0.000298,0.002413,-0.006735,-0.005464,0.000113,1.000000,-0.000275,0.000694,-0.006127
Soil Type,0.002831,-0.007621,0.003921,-0.003941,-0.000650,0.005314,-0.000275,1.000000,-0.006213,0.016576
Crop Type,-0.000953,-0.006753,0.002414,-0.003400,0.001611,-0.003748,0.000694,-0.006213,1.000000,-0.013041
Fertilizer Name,-0.000179,0.001727,0.010626,0.011493,-0.004725,-0.010872,-0.006127,0.016576,-0.013041,1.000000


In [16]:
X = df_train[["Temparature","Humidity","Moisture","Crop Type","Soil Type"]]

In [17]:
y = df_train[["Fertilizer Name"]]

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [19]:
import torch
import torch.nn as nn
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import Dataset

class FeatureTokenizer(nn.Module):
    def __init__(self, num_numeric, cat_cardinalities, embed_dim):
        super().__init__()
        self.num_linear = nn.Linear(num_numeric, embed_dim)
        self.cat_embeds = nn.ModuleList([
            nn.Embedding(cardinality, embed_dim) for cardinality in cat_cardinalities
        ])
        self.cls_token = nn.Parameter(torch.randn(1, 1, embed_dim))

    def forward(self, x_num, x_cat):
        # Sanity check
        assert x_cat.shape[1] == len(self.cat_embeds), (
            f"x_cat has {x_cat.shape[1]} columns, but cat_embeds has {len(self.cat_embeds)} embeddings"
        )

        # Project numerical features
        num_tokens = self.num_linear(x_num).unsqueeze(1)  # [B, 1, D]

        # Clamp categorical indices to valid range
        x_cat_clamped = torch.stack([
            torch.clamp(x_cat[:, i], max=self.cat_embeds[i].num_embeddings - 1)
            for i in range(x_cat.shape[1])
        ], dim=1)

        # Embed categorical features
        cat_tokens = [embed(x_cat_clamped[:, i])[:, None, :] for i, embed in enumerate(self.cat_embeds)]

        # Concatenate [CLS] + categorical + numerical tokens
        tokens = torch.cat(
            [self.cls_token.expand(x_num.size(0), -1, -1)] + cat_tokens + [num_tokens],
            dim=1
        )  # [B, T, D]

        return tokens

In [20]:
from torch.utils.data import Dataset
class FTTransformer(nn.Module):
    def __init__(self, num_numeric, cat_cardinalities, embed_dim=32, num_heads=4, num_layers=2, num_classes=7):
        super().__init__()
        cat_cardinalities = [
    df["Soil Type"].nunique(),
    df["Crop Type"].nunique()]
        self.tokenizer = FeatureTokenizer(num_numeric=3, cat_cardinalities=cat_cardinalities, embed_dim=embed_dim)
        encoder_layer = nn.TransformerEncoderLayer(d_model=embed_dim, nhead=num_heads, batch_first=True)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x_num, x_cat):
        tokens = self.tokenizer(x_num, x_cat)          # includes [CLS] token already
        encoded = self.transformer(tokens)
        return self.head(encoded[:, 0, :])     

In [21]:
class FTTransformer(nn.Module):
    def __init__(self, num_numeric, cat_cardinalities, embed_dim=64, num_heads=4, num_layers=4, num_classes=7):
        super().__init__()
        self.tokenizer = FeatureTokenizer(num_numeric, cat_cardinalities, embed_dim)
#PyTorch uses the Post-LN Transformer architecture:
#It adds residual connections around both the multi-head attention and feedforward blocks.
#It applies layer normalization after each residual connection.

        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            batch_first=True,
            activation='gelu'  # smoother gradients than ReLU
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x_num, x_cat):
        tokens = self.tokenizer(x_num, x_cat)  # [B, T, D] with [CLS] already included
        encoded = self.transformer(tokens)
        return self.head(encoded[:, 0, :])     # pull from [CLS] token

In [22]:
import torch
import torch.nn as nn
from torch_geometric.nn import GATConv
from torch.nn import TransformerEncoderLayer, TransformerEncoder
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
from torch.utils.data import Dataset
class GATTransformer(nn.Module):
    def __init__(self, num_numeric, cat_cardinalities, embed_dim=64, num_heads=4, num_layers=2, num_classes=7):
        super().__init__()

        self.gat = GATConv(embed_dim, embed_dim, heads=1, concat=False)
        self.tokenizer = FeatureTokenizer(num_numeric, cat_cardinalities, embed_dim)

        encoder_layer = TransformerEncoderLayer(
            d_model=embed_dim,
            nhead=num_heads,
            batch_first=True
        )
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_layers)

        self.head = nn.Sequential(
            nn.LayerNorm(embed_dim),
            nn.Linear(embed_dim, num_classes)
        )

    def forward(self, x_numeric, x_cat, edge_index):  # 👈 Now inside the class!
        x_token = self.tokenizer(x_numeric, x_cat)        # [batch, seq_len, embed_dim]
        x_encoded = self.transformer(x_token)             # [batch, seq_len, embed_dim]
        x_pooled = x_encoded.mean(dim=1)                  # [batch, embed_dim]
        x_graph = self.gat(x_pooled, edge_index)          # [nodes, embed_dim]
        out = self.head(x_graph)                          # [nodes, num_classes]
        return out

In [23]:
numeric_features = ["Temperature", "Humidity", "Moisture"]  # update as needed
num_numeric = len(numeric_features)

In [24]:
cat_cardinalities = [
    df["Soil Type"].nunique(),
    df["Crop Type"].nunique()
]
# Encode string categories if not already done
df["Soil Type"] = df["Soil Type"].astype("category").cat.codes
df["Crop Type"] = df["Crop Type"].astype("category").cat.codes
df["Fertilizer Name"] = df["Fertilizer Name"].astype("category").cat.codes
# Create the categorical tensor
x_cat_tensor = torch.tensor(df[["Soil Type", "Crop Type"]].values, dtype=torch.long)
y_tensor = torch.tensor(df[["Fertilizer Name"]].values, dtype=torch.long)

In [25]:

class FertilizerDataset(Dataset):
    def __init__(self, x_num, x_cat, y):
        self.x_num = x_num
        self.x_cat = x_cat
        self.y = y  # y must be a LongTensor

    def __len__(self):
        return len(self.y)

    def __getitem__(self, idx):
        # ✅ Return three tensors — no tuple wrapping!
        return self.x_num[idx], self.x_cat[idx], self.y[idx]

In [26]:
# Features
x_cat_tensor = torch.tensor(df[["Soil Type", "Crop Type"]].values, dtype=torch.long)
x_num_tensor = torch.tensor(df[["Temparature", "Humidity", "Moisture"]].values, dtype=torch.float32)
# Target
y_train = torch.tensor(df["Fertilizer Name"].values, dtype=torch.long)

In [27]:
y_tensor = y_train.squeeze()  # shape: [750000]

In [28]:
model = GATTransformer(
    
    num_numeric=num_numeric,
    cat_cardinalities=cat_cardinalities,
    embed_dim=8,
    num_heads=4,
    num_layers=4,
    num_classes=7
).to(device)

In [29]:
x_fused = torch.cat([x_num_tensor, x_cat_tensor], dim=1)  # Final GCN input

In [30]:
from sklearn.neighbors import NearestNeighbors
import torch
import numpy as np

# Convert to NumPy
X_np = x_fused.detach().cpu().numpy()

# Fit k-NN model
nbrs = NearestNeighbors(n_neighbors=5, algorithm='ball_tree').fit(X_np)
_, indices = nbrs.kneighbors(X_np)

# Build edge_index
rows = np.repeat(np.arange(X_np.shape[0]), indices.shape[1])
cols = indices.flatten()

edge_index_np = np.stack([rows, cols], axis=0)  # shape: [2, num_edges]
edge_index = torch.from_numpy(edge_index_np).long()

In [ ]:
model.eval()
with torch.no_grad():
    out = model(x_num_tensor.to(device), x_cat_tensor.to(device), edge_index.to(device))
    _, preds = torch.max(out, dim=1)
    correct = (preds == y_tensor.to(device).squeeze()).sum().item()
    accuracy = correct / y_tensor.size(0)

print(f"✅ Overall Dataset Accuracy: {accuracy * 100:.2f}%")